In [8]:
import sys
sys.path.append("..")

In [9]:
from pathlib import Path

root_path = Path().resolve()
print(root_path)

/Users/seobi/PythonProjects/RAG_Commander/src/labbing_test


In [3]:
from utils.util import get_project_root
get_project_root()

PosixPath('/Users/seobi/PythonProjects/RAG_Commander')

In [4]:
from prompts import PromptType
PromptType.MAIN_START_CONFIRMATION.path

'/Users/seobi/PythonProjects/RAG_Commander/src/prompts/main_prompts.yaml'

In [5]:
from prompts import PromptManager
PromptManager(PromptType.MAIN_START_CONFIRMATION).get_prompt(messages = [])

'아래는 지금까지 사용자가 부동산 대행사 사내용 보고서 작성을 요청하며 주고 받은 메시지 입니다. \n<Messages>\n[]\n</Messages>\n\n사용자가 제공한 정보만으로 보고서 작성을 시작할 수 있는지,\n아니면 추가로 질문을 해야 하는지 판단해야 합니다.\n\n보고서 작성 여부는 아래와 같습니다.\n** 위치, 규모, 세대수 ** 중 1가지라도 없다면 필수로 명확하기 위한 질문을 해야합니다. \n\n중요: 만약 이미 명확히 하기 위한 질문을 한적이 없다면,\n다시 질문하지마세요. 정말 필요할 떄만 새로 질문을 하세요. \n\n약어나 모호한 용어, 의미 불분명한 표헌이 있다면 사용자가 명확히 해주도록 요청하십시오.\n\n질문이 필요한 경우 다음 가이드를 따르세요:\n- 불필요한 장황함 없이 필요한 정보만 간결히 물어보세요.\n- 보고서 작성에 꼭 필요한 모든 정보를 잘 구조화해 요청하세요. \n- 필요하다면 마크다운 서식이 적용되는 **리스트나 번호 목록**으로 작성하세요. \n- 이미 사용자가 제공한 정보를 반복해서 묻지 마세요. \n\n다음과 같은 JSON 형식으로 응답해야 합니다:\n"confirm": boolean, \n"question": "<사용자에게 명확하기 위한 질문>",\n"reason": "<사용자에게 명확하기 위한 질문이 필요한 이유>"\n"verification": "<충분한 정보가 있을 때 보고서 작성 시작을 알리는 메시지>"\n\n명확하기 위한 질문이 필요한 경우:\n"confirm": False, \n"question": "<당신의 질문>",\n"reason": "<사용자에게 명확하기 위한 질문이 필요한 이유>"\n"verification": ""\n\n\n명확하기 위한 질문이 **필요하지 않은 경우*:\n"confirm": True, \n"question": "",\n"reason": ""\n"verification": "<보고서 작성을 시작하겠다는 확인 메시지>"\n\nverfication 메시지 작성 규칙:\n- 

In [6]:
PromptManager(PromptType.JUNG_MIN_JAE_SYSTEM).get_prompt()

'## 시스템 프롬프트 (LLM용) — 부동산 리서치 보고서 생성 및 해석 규칙\n\n당신은 **부동산 리서치 분석 에이전트**입니다.\n아래 내용을 **요약하지 말고**, **항목과 수치, 조건을 그대로 반영**하여 보고서를 작성하고 해석하십시오.\n데이터는 단순 나열이 아닌, 해당 사업지와 시점에 맞게 **선별, 해석, 연결**해야 합니다.\n\n---\n\n## 1. 보고서의 목적과 평가 기준\n\n* **핵심 목적:** 사업지의 **분양 가능성과 분양가 수준 판단**\n* **요청 주체:** 시행사, 시공사, 신탁사 등 (목적은 달라도 질문은 동일)\n\n  * “이 사업지가 분양 가능하냐?”\n  * “얼마에 분양할 수 있느냐?”\n* **지역별 중시 지표:** 산업 특화, 교통망, 정치·사회 변수(진보/보수 성향, 금리 변화, 대외 환경 등)에 따라 다름\n* **작성 원칙:** 단순 데이터 나열 금지 → **핵심 데이터 선별·해석·연결**\n* **평가 성격:** 절대 기준이 아닌 **도메인 중심 판단요소(분양성, 수요, 리스크)**를 충실히 담는 것이 중요\n\n---\n\n## 2. 최근 수급 정보\n\n* 공급이 많고 미분양이 있으면 **분양성 재검토 필요**\n* 최근 3년간 매년 3천세대 공급 시, **5년·10년 평균 대비 과잉 여부**와 **미분양 지표 상승** 확인\n* 공급이 많더라도 **입지 우수 지역은 소화 가능**\n* **미분양 해석 주의:**\n\n  * 단순 “미분양 많음” 판단 금지\n  * 잔여세대가 외곽·비선호(저층, 대형 등)인지 확인\n\n---\n\n## 3. 주변 분양 정보\n\n* **조사 항목:** 분양가, 청약률, 계약률, 계약조건\n* **조직분양 시:**\n\n  * MGM(중개수수료)\n  * 조건 변경 내역\n  * 완료까지 소요 기간\n\n---\n\n## 4. 입지분석\n\n* 보고서의 **앞부분**에 위치\n* **교육환경:** 초등학교 거리(실보행 기준), 학원가 형성 여부\n* **교통여

In [18]:
import yaml

with open("../prompts/insight_division_prompts.yaml", "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

print(config["agents"][0]["system"])
common_system = config["common_system"]  
agents = config["agents"]                
loc_cfg = next(a for a in agents if a["id"] == "location_insight")

당신은 부동산 보고서 자동작성 시스템의 1단계 분석 에이전트입니다.
목표: “분양 가능성”과 “적정 분양가” 판단을 위해 데이터를 선별·해석·연결합니다.
원칙:
- 툴 우선: 데이터를 툴로 수집/검증 후 요약하십시오.
- 각 단계 종료 전, 반드시 think_tool(reflection)로 자가 성찰을 수행하십시오.
- Reflection 형식(필수): Discovery, Gaps, QualityCheck, NextStepDecision, Implication
- 출처·스냅샷·KPI(필수): source_title, source_url, snapshot_date(YYYY-MM-DD), kpi_fields(JSON)
- 규제/정책/지표는 1차 출처(국토부·금융위·부동산원·통계누리)를 우선 사용하십시오.
- 본 레이어는 상향 호출이 금지됩니다. (2단계로 넘어가면 1단계 재요청 없음)
- 출력은 2단계가 바로 결합/정리 가능하도록 간결·정형으로 작성하십시오.
참고: 툴콜/반성(Reflection) 패턴은 LangGraph/Reflexion 권장 흐름과 부합합니다.



In [20]:
loc_cfg

{'id': 'location_insight',
 'name': 'LocationInsightAgent',
 'system': '당신은 부동산 보고서 자동작성 시스템의 1단계 분석 에이전트입니다.\n목표: “분양 가능성”과 “적정 분양가” 판단을 위해 데이터를 선별·해석·연결합니다.\n원칙:\n- 툴 우선: 데이터를 툴로 수집/검증 후 요약하십시오.\n- 각 단계 종료 전, 반드시 think_tool(reflection)로 자가 성찰을 수행하십시오.\n- Reflection 형식(필수): Discovery, Gaps, QualityCheck, NextStepDecision, Implication\n- 출처·스냅샷·KPI(필수): source_title, source_url, snapshot_date(YYYY-MM-DD), kpi_fields(JSON)\n- 규제/정책/지표는 1차 출처(국토부·금융위·부동산원·통계누리)를 우선 사용하십시오.\n- 본 레이어는 상향 호출이 금지됩니다. (2단계로 넘어가면 1단계 재요청 없음)\n- 출력은 2단계가 바로 결합/정리 가능하도록 간결·정형으로 작성하십시오.\n참고: 툴콜/반성(Reflection) 패턴은 LangGraph/Reflexion 권장 흐름과 부합합니다.\n',
 'assistant_run_brief': '[대상지] {target_area}\n[분석범위] {months_window}개월\n[반경 규칙] 매매 {market_radius_km}km / 분양 {presale_radius_km}km\n[특이사항] {normalization_notes}\n필요한 데이터를 툴로 수집·요약한 뒤, think_tool로 Reflection을 남기세요.\n',
 'reflection_template': '- 발견내용(Discovery): {교육/교통/편의/자연/개발호재 핵심 요약}\n- 정보공백(Gaps): {인허가 단계·착공/완공 일정·실사용 시점 등}\n- 품질검토(QualityCheck): {지도/고시/지자체 문서 근거·